In [148]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet # To provide a best estimate of the encoding that was used in the text data
import io # For string operations
%matplotlib inline

# Libraries for data preparation and model building
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, TweetTokenizer, TreebankWordTokenizer
from nltk.corpus import stopwords
import string
import datetime
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import math
import re
from sklearn.utils import resample
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn import feature_selection
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif, mutual_info_classif,chi2, VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from gensim.models import Word2Vec
import multiprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.model_selection import cross_val_predict

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE,SMOTENC
from imblearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
# LOAD DATASET

def transform_categorical_labels(data):
    # Dictionary mapping numerical categories to labels
    label_map = {
        2: 'News',
        1: 'Pro',
        0: 'Neutral',
        -1: 'Anti'
    }
    transformed_data = data.map(label_map)
    return transformed_data

def class_distribution(data):
    if isinstance(data, pd.DataFrame):
        unique_classes, class_counts = data.iloc[:, 0].value_counts().index, data.iloc[:, 0].value_counts().values
    elif isinstance(data, pd.Series):
        unique_classes, class_counts = data.value_counts().index, data.value_counts().values
    class_dict = dict(zip(unique_classes, class_counts))
    return class_dict

def load_dataset(environment, size, change_labels):
    if environment == 'colab':
        from google.colab import drive
        drive.mount('/content/drive')
        csv_file = '/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/data/train.csv'
    else:
        csv_file = r'G:\My Drive\Professionele ontwikkeling\Data Science\Explore Data Science Course\Sprint 6_Advanced Classification\Predict\advanced-classification-predict\data\train.csv'
    df = pd.read_csv(csv_file)
    if change_labels:
        df['sentiment'] = transform_categorical_labels(df['sentiment'])
    sample_size = int(len(df) * size)
    if size == 1:
        pass
    else:
        X = df.drop(columns=['sentiment']).copy()
        y = df.sentiment.copy()
        X_sample, _, y_sample, _ = train_test_split(X, y, train_size=sample_size, stratify=y, random_state=42)
        df = pd.concat([X_sample, y_sample], axis=1)

#       np.random.seed(42)  # OLD CODE
#       pd.read_csv(csv_file).shape[0] - n  # OLD CODE
#        df = pd.read_csv(csv_file, skiprows=lambda i: i > 0 and np.random.rand() > n / (i + 1), nrows=n)  # OLD CODE

    class_dict = class_distribution(df['sentiment'])

    return df, class_dict


# def load_full_dataset():  # OLD CODE
#   df = pd.read_csv(csv_file)  # OLD CODE
#   return df, 'full'  # OLD CODE


In [59]:
class NoiseRemover(BaseEstimator, TransformerMixin):
    def __init__(self, switch=None):
        self.switch = switch

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        if self.switch == 'off':
            X_transformed = X
        elif self.switch == 'on':
            pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'   # Find all hyperlinks
            pattern_handle = r'@[A-Za-z0-9_]+'
            subs_url = r''
            subs_handle = r''
            X_transformed = pd.Series(X).replace(to_replace=pattern_url, value=subs_url, regex=True)
            X_transformed = X_transformed.replace(to_replace=pattern_handle, value=subs_handle, regex=True)
        else:
            raise ValueError("Invalid switch. Choose either 'on' or 'off'.")
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [60]:
class EmoticonConverter(BaseEstimator, TransformerMixin):
    def __init__(self, switch=None):
        self.switch = switch

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        if self.switch == 'off':
            X_transformed = X
        elif self.switch == 'on':
            emoticon_dictionary = {':\)': 'smiley_face_emoticon',
                                   ':\(': 'frowning_face_emoticon',
                                   ':D': 'grinning_face_emoticon',
                                   ':P': 'sticking_out_tongue_emoticon',
                                   ';\)': 'winking_face_emoticon',
                                   ':o': 'surprised_face_emoticon',
                                   ':\|': 'neutral_face_emoticon',
                                   ':\'\)': 'tears_of_joy_emoticon',
                                   ':\'\(': 'crying_face_emoticon'}
            X_transformed = X.replace(emoticon_dictionary, regex=True)
        else:
            raise ValueError("Invalid switch. Choose either 'on' or 'off'.")
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [61]:
class PunctuationRemover(BaseEstimator, TransformerMixin):
    def __init__(self, switch=None):
        self.switch = switch

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        def expand_contractions(text):
            contractions = {"'t": " not","'s": " is","'re": " are","'ll": " will", "'m": " am"}
            pattern = re.compile(r"\b(" + "|".join(re.escape(key) for key in contractions.keys()) + r")\b")
            text = re.sub(r"n't\b", " not", text) # Replace "n't" with " not"
            text = pattern.sub(lambda match: contractions[match.group(0)], text) # Replace all other contractions except for "n't"
            return text

        def remove_punctuation(text):
            return ''.join([l.lower() for l in text if l not in string.punctuation])

        if self.switch == 'off':
            X_transformed = X
        elif self.switch == 'on':
            X_transformed = X.apply(lambda x: expand_contractions(x))
            X_transformed = X_transformed.apply(lambda x: remove_punctuation(x))
        else:
            raise ValueError("Invalid switch. Choose either 'on' or 'off'.")
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [62]:
class Tokenizer(BaseEstimator, TransformerMixin):
    def __init__(self, type=None):
        self.type = type

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.type == 'TweetTokenizer':
            tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True)
        elif self.type == 'TreebankWordTokenizer':
            tokenizer = TreebankWordTokenizer()
        else:
            raise ValueError("Invalid tokenizer type. Choose either 'TweetTokenizer' or 'TreebankWordTokenizer'")
        X_transformed = X.apply(tokenizer.tokenize)
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [70]:
class GarbageRemover(BaseEstimator, TransformerMixin):
    def __init__(self, switch=None):
        self.switch = switch
        self.pattern = re.compile(r'^[a-zA-Z\U0001F300-\U0001FAD6\U0001F004-\U0001F251]*[^a-zA-Z\U0001F300-\U0001FAD6\U0001F004-\U0001F251]+[a-zA-Z\U0001F300-\U0001FAD6\U0001F004-\U0001F251]*$')
    def is_garbage(self, word):
        """
        Define a regular expression pattern to match garbage words
        Garbage is defined as words starting with anything but a letter, or
        words containing strange characters.  Emojis are not considered garbage
        """
        return bool(self.pattern.match(word))

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.switch == 'off':
            return X
        elif self.switch == 'on':
            transformed_words = [[word for word in word_list if not self.is_garbage(word)] or ['blankpost'] for word_list in X]
            return pd.Series(transformed_words)
        else:
            raise ValueError("Invalid switch. Choose either 'on' or 'off'.")

    def fit_transform(self, X, y=None):
        if self.switch == 'off':
            return X
        elif self.switch == 'on':
            self.fit(X, y)
            return self.transform(X, y)
        else:
            raise ValueError("Invalid switch. Choose either 'on' or 'off'.")

In [13]:
class StopwordRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_transformed = X.apply(lambda tokens: [t for t in tokens if t.lower() not in self.stop_words])
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [14]:
class Lemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, pos=None):
        self.pos = pos

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.pos not in ['v','n', 'a', 'r']:
            raise ValueError('Invalid pos type. Choose either a, n, r or v')
        else:
            lemmatizer = WordNetLemmatizer()
            X_transformed = X.apply(lambda tokens: [lemmatizer.lemmatize(word, pos=self.pos) for word in tokens])
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [15]:
class Vectorize(BaseEstimator, TransformerMixin):
    def __init__(self, type=None, max_df=None, min_df=None, ngram_range=None, max_features=None):
        self.type = type
        self.max_df = max_df
        self.min_df = min_df
        self.ngram_range = ngram_range
        self.max_features = max_features

    def fit(self, X, y=None):
        if self.type == 'count':
            self.vectorizer = CountVectorizer(max_features=self.max_features, lowercase=True,
                                              max_df=self.max_df, min_df=self.min_df,
                                              ngram_range=self.ngram_range)
        elif self.type == 'tfidf':
            self.vectorizer = TfidfVectorizer(max_features=self.max_features, lowercase=True,
                                              max_df=self.max_df, min_df=self.min_df,
                                              ngram_range=self.ngram_range)
        else:
            raise ValueError("Invalid vectorizer type. Choose either 'count' or 'tfidf'")

        X_joined = [' '.join(tokens) for tokens in X]
        self.vectorizer.fit(X_joined)
        return self.vectorizer.fit(X_joined)

    def transform(self, X, y=None):
        X_joined = [' '.join(tokens) for tokens in X]
        return self.vectorizer.transform(X_joined)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [147]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, type=None, percentile=None,threshold=None):
        self.type = type
        self.percentile = percentile
        self.threshold = threshold

    def fit(self, X, y=None):
        if self.type == 'chi2':
            self.selector = SelectPercentile(score_func=chi2, percentile=self.percentile)
        elif self.type == 'anova_f':
            self.selector = SelectPercentile(score_func=f_classif, percentile=self.percentile)
        elif self.type == 'variance_threshold':
            self.selector = VarianceThreshold(threshold=self.threshold)
        else:
            raise ValueError("Invalid selector type. Choose between 'chi2','anova_f' or 'variance_threshold'.")
        self.selector.fit(X, y)
        return self

    def transform(self, X, y=None):
        return self.selector.transform(X)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

In [222]:
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, type=None):
        self.type = type
        self.scaler = None

    def fit(self, X, y=None):
        if self.type == 'none':
            return self
        else:
            if self.type == 'robust':
                self.scaler = RobustScaler(with_centering=False)
            elif self.type == 'maxabs':
                self.scaler = MaxAbsScaler()
            elif self.type == 'standard':
                self.scaler = StandardScaler(with_mean=False)
            else:
                raise ValueError("Invalid scaler type. Choose between 'none, 'standard', 'robust' or 'maxabs'.")
            return  self.scaler.fit(X)

    def transform(self, X, y=None):
        if self.type == 'none':
            return X
        else:
            return self.scaler.transform(X)

    def fit_transform(self, X, y=None):
        if self.type == 'none':
            return X
        else:
            self.fit(X, y)
            return self.transform(X, y)

In [ ]:
X = df_train['message']
y = df_train['sentiment']
noise = NoiseRemover(switch='on')
X = noise.fit_transform(X)
emoti = EmoticonConverter(switch='on')
X = emoti.fit_transform(X)
punc = PunctuationRemover(switch='on')
X = punc.fit_transform(X)
token = Tokenizer(type='TweetTokenizer')
X = token.fit_transform(X)
garbage = GarbageRemover(switch='on')
X = garbage.fit_transform(X)
stop = StopwordRemover()
X = stop.fit_transform(X)
lemma = Lemmatizer(pos='v')
X = lemma.fit_transform(X)
vector = Vectorize(type='tfidf', max_df= 0.75, min_df=1, ngram_range=(1,1),max_features=None)
X = vector.fit_transform(X)
y_encoded = pd.get_dummies(y, columns=['sentiment'])
mutual_info = mutual_info_classif(X, y)
print(mutual_info)
#smote = SMOTE()
#X, y = smote.fit_resample(X, y)
#print(X.shape)
##print(y.shape)
#fselect = MutualInformationSelector(percentage=0.1)
#X = fselect.fit_transform(X, y)
print(X)

"""
words_list = X.explode().tolist()



from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_word_cloud(words, title):
    # Join the words into a single string
    text = ' '.join(words)

    # Generate a word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    # Plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

create_word_cloud(words_list, "All Words")

def classify_words(words):
    actual_words = []
    garbage_words = []

    # Regular expression pattern to match a word
    word_pattern = re.compile(r'^[a-zA-Z]+$')

    for word in words:
        # Check if the word matches the pattern
        if word_pattern.match(word) and word:
            actual_words.append(word)
        else:
            garbage_words.append(word)

    return actual_words, garbage_words

# Example usage:
actual, garbage = classify_words(words_list)
print(f'Actual words: {len(actual)}')
print(f'Garbage words: {len(garbage)}')

create_word_cloud(actual, "Actual Words")
create_word_cloud(garbage, "Garbage")

import csv


def is_garbage(word):
    # Define a regular expression pattern to match garbage words
    pattern = r'^[a-zA-Z\U0001F300-\U0001FAD6\U0001F004-\U0001F251]*[^a-zA-Z\U0001F300-\U0001FAD6\U0001F004-\U0001F251]+[a-zA-Z\U0001F300-\U0001FAD6\U0001F004-\U0001F251]*$'
    # Check if the word matches the pattern
    return bool(re.match(pattern, word))

def count_garbage_words(words):
    garbage_count = {}

    for word in words:
        if is_garbage(word):
            if word in garbage_count:
                garbage_count[word] += 1
            else:
                garbage_count[word] = 1

    return garbage_count

def export_to_csv(garbage_counts, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Garbage Word', 'Count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for word, count in garbage_counts.items():
            writer.writerow({'Garbage Word': word, 'Count': count})

# Example usage:
garbage_counts = count_garbage_words(words_list)

# Export to CSV
export_to_csv(garbage_counts, '/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/garbagewords.csv')


def words_summary(words):
    summary = [0] * (max(len(word) for word in words) + 1)

    # Iterate through the words
    for word in words:
        # Get the length of the word
        length = len(word)
        # Increment the count for the corresponding length
        summary[length] += 1

    return summary

# Example usage:
summary = words_summary(words_list)

# Print the summary
print("Summary of words per length:")
for length, count in enumerate(summary):
    if count > 0:
        print(f"Length {length}: {count} words")
"""


In [242]:
size = 0.2
change_labels = True
df_train, class_dictionary = load_dataset('colab',size,change_labels=change_labels)
# Next section is to calculate class imbalance, for us in randomover and randomundersampling

number_classes = len(class_dictionary)
majority = max(class_dictionary.values())
average_class_count = int(size / number_classes)
oversampling_strategy = {key: max(average_class_count, count)  for key, count in class_dictionary.items()}
undersampling_strategy = {key: average_class_count  for key, count in oversampling_strategy.items()}

df_train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,message,tweetid,sentiment
7669,RT @Mrfarrago: “catastrophic man-made global w...,753059,Anti
15577,"In executive order, Trump to dramatically chan...",24585,News
14132,The U.S. public is largely skeptical of climat...,916197,News
3116,Roses you carnation pink All global warming re...,353744,Neutral
5074,RT @StephenSchlegel: she's thinking about how ...,465734,Pro


In [253]:
X_train, X_test, y_train, y_test = train_test_split(df_train['message'], df_train['sentiment'], test_size=0.2, random_state=42, stratify=df_train['sentiment'])
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')
print(f'Distribution of y_train: {class_distribution(y_train)}')
print(f'Distribution of y_test: {class_distribution(y_test)}')

# Defining preprocessing steps
preprocessing_steps = [
                       ('noise_removal', NoiseRemover()),
                       ('emoticon_convertion', EmoticonConverter()),
                       ('punctuation_removal', PunctuationRemover()),
                       ('tokenization', Tokenizer()),
                       ('garbageout', GarbageRemover()),
                       ('stopword_removal', StopwordRemover()),
                       ('lemmatization', Lemmatizer()),
                       ('vectorization', Vectorize()),
                       #('oversampler', RandomOverSampler(sampling_strategy=oversampling_strategy, random_state=42)),
                       #('undersampler', RandomUnderSampler(sampling_strategy=undersampling_strategy, random_state=42)),
                       ('smote', SMOTE()),
                       ('feature_selection', FeatureSelector()),
                       ('scaler', Scaler()),
                       ]


# Create the pipelines
#model = xgb.XGBClassifier()
#model = AdaBoostClassifier()
#model_name = 'Adaboost'
#model = MultinomialNB()
#model_name = 'MultinomialNB'
model = LogisticRegression()
model_name = 'LogisticRegression'
#model = KNeighborsClassifier()
#model_name = 'KNN'
#model = SVC()
#model_name='SVC RBF'
#model = RandomForestClassifier()
#model_name='RandomForest'
#model = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', alpha=0.0001,
#                    batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
#                    max_iter=200, random_state=42)
"""
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # 3 classes, change as per your number of classes
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# Wrap Keras model in a scikit-learn compatible classifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=10, verbose=0)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
# Convert integers to one-hot encoded format
y_train_one_hot = to_categorical(y_train_encoded)
y_test_one_hot = to_categorical(y_test_encoded)
y_train = y_train_one_hot
yest = y_test_one_hot
"""
pipeline = Pipeline(preprocessing_steps  + [('model', model)])

# Define parameter grid for GridSearchCV
param_grid = {
    'noise_removal__switch': ['off'],#'on'],
    'emoticon_convertion__switch': ['off'],#,'on'],
    'punctuation_removal__switch':  ['off'],#,'on'],
    'tokenization__type': ['TreebankWordTokenizer'],#'TweetTokenizer'],
    'garbageout__switch': ['off'],#,'on'],
    'lemmatization__pos': ['v'],#,'n','a','r'],
    'vectorization__type': ['tfidf'],#,'count'],
    'vectorization__max_df': [0.75],#
    'vectorization__min_df': [1],#, 2],
    'vectorization__ngram_range': [(1,1)],# (1, 10)],
    'vectorization__max_features': [None],
    'smote__k_neighbors': [3],
    'smote__random_state': [42],
    'feature_selection__percentile': [75],#,25],
    'feature_selection__type': ['chi2'],#'variance_threshold'],
    'feature_selection__threshold': [0.001],
    #'feature_selection__percentage': [0.10, 0.75],
    'scaler__type': ['robust'],  # 'none','maxabs'],


             # Hyperparameters for the model

              #'model__n_neighbors': [3,4,5],                                #KNN
              #'model__weights': [ 'distance','uniform'],               #KNN
              #'model__metric': ['euclidean', 'manhattan'],            #KNN
              #'model__C': [100,10,1],                                  #SVM RBF
              #'model__gamma': [0.1,0.001, 0.01,1],# 0.1]                                 #SVM RBF
               'model__penalty': ['l2'],#,'l1'],                            #LogisticRegression
               'model__C': [7],#,100,1,0.001,0.1,],                             #LogisticRegression
               'model__solver': ['liblinear'],#['sag']                     #LogisticRegression
               'model__multi_class': ['ovr'],#'multinomial',               #LogisticRegression
               'model__max_iter': [100000],                               #LogisticRegression
               'model__tol': [0.0001],#, 0.0001],                                #LogisticRegression
               #'model__n_jobs': [-1],                                           #LogisticRegression
               'model__random_state': [42],                                     #LogisticRegression
               #'model__alpha': [0.1],#3, 0.5,],# 0.1,1,5,10],                               #Multinomial NB
               #'model__fit_prior': [True],# False],                               #Multinomial NB
               #'model__n_estimators': [500],                              #Adaboost
               #'model__learning_rate': [0.1],#, 1, 5],                              #Adaboost
               #'model__random_state': [42],                                     #Adaboost
               #'model__algorithm': ['SAMME'],                                    #Adaboost
               #'model__estimator': [LogisticRegression()]#,SVC(kernel='rbf', C=100,gamma=0.1)],DecisionTreeClassifier(), ],   #Adaboost

                #'model__n_estimators': [100],#200],                               #XGBoost
                #'model__max_depth': [30],#,6],                               #XGBoost
                #'model__learning_rate': [0.3, 1],                               #XGBoost
                #'model__subsample': [1.0],                               #XGBoost
              }

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro', error_score='raise', verbose=2)
grid_search.fit(X_train, y_train)
df_gridsearch = pd.DataFrame(grid_search.cv_results_)
print(grid_search.best_score_)
print(grid_search.best_params_)

selected_columns = ['Model','Dataset','Timestamp','mean_fit_time','mean_score_time','change_labels','params','split0_test_score','split1_test_score',
                    'split2_test_score','split3_test_score','split4_test_score',
                    'std_test_score','mean_test_score']

#Create blank results dataframe - only do this once
###results_df = pd.DataFrame(columns=selected_columns)
###results_df.to_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/Model results.csv', index=False)
new_data = pd.DataFrame(columns=selected_columns)

new_data[['mean_fit_time','mean_score_time','params','split0_test_score','split1_test_score',
         'split2_test_score','split3_test_score','split4_test_score',
          'std_test_score','mean_test_score']] = df_gridsearch[['mean_fit_time',
         'mean_score_time','params','split0_test_score','split1_test_score',
         'split2_test_score','split3_test_score','split4_test_score',
         'std_test_score','mean_test_score']]
new_data['Model'] = model_name
new_data['Dataset'] = size
new_data['Timestamp']= datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
new_data['change_labels'] = change_labels
new_data.to_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/Model results.csv', mode='a', header=False, index=False)


# Do a stand-alon pipeline to see the full class report
#pipeline.fit(X_train, y_train)
#y_pred = pipeline.predict(X_test)
#report = classification_report(y_test, y_pred)
#print("Classification Report for test set:")
#print(report)
#print('************************')

#k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
# Perform k-fold cross-validation and generate predictions
#predictions = cross_val_predict(pipeline, X_train, y_train, cv=k_fold)

# Generate classification report
#report = classification_report(y_train, predictions)

# Print the classification report
#print("Classification Report for CV:\n", report)



Shape of X_train: (2530,)
Shape of y_train: (2530,)
Shape of X_test: (633,)
Shape of y_test: (633,)
Distribution of y_train: {'Pro': 1365, 'News': 582, 'Neutral': 376, 'Anti': 207}
Distribution of y_test: {'Pro': 341, 'News': 146, 'Neutral': 94, 'Anti': 52}
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END emoticon_convertion__switch=off, feature_selection__percentile=75, feature_selection__threshold=0.001, feature_selection__type=chi2, garbageout__switch=on, lemmatization__pos=v, model__C=7, model__max_iter=100000, model__multi_class=ovr, model__penalty=l2, model__random_state=42, model__solver=liblinear, model__tol=0.0001, noise_removal__switch=off, punctuation_removal__switch=off, scaler__type=robust, smote__k_neighbors=3, smote__random_state=42, tokenization__type=TreebankWordTokenizer, vectorization__max_df=0.75, vectorization__max_features=None, vectorization__min_df=1, vectorization__ngram_range=(1, 1), vectorization__type=tfidf; total time=   4.1s
[CV] END emo

In [ ]:
print(grid_search.best_params_)

In [ ]:
scorer = make_scorer(f1_score, average='macro')
f1_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=scorer)
print("Mean F1 macro score:", f1_scores.mean())
pipeline.fit(X_train, y_train)
model_stats = pipeline.named_steps['model']

# Obtain the number of features used by the model
attributes = model_stats.__dict__

# Print all attributes
for attr, value in attributes.items():
    print(attr, ":", value)


In [19]:
# MODEL COMPARISONS

# Load the data
label_change = True
df_train, size, class_dictionary = load_dataset('colab',0.2,change_labels=label_change)
X_train, X_test, y_train, y_test = train_test_split(df_train['message'], df_train['sentiment'], test_size=0.2, random_state=42, stratify=df_train['sentiment'])

# Set common preprocessing parameters to include in GridSearch
preprocessing_params = {
    'tokenization__type': ['TweetTokenizer','TreebankWordTokenizer'],
    'lemmatization__pos': ['v'],
    'vectorization__type': ['count','tfidf'],
    'vectorization__max_df': [0.25,0.5,0.75],
    'vectorization__min_df': [1],
    'vectorization__ngram_range': [(1,1),(1,3)],
    'vectorization__max_features': [None],
    'scaler__type': ['robust'],
    'smote__k_neighbors': [3],
    'smote__random_state': [42],
    'feature_selection__percentile': [10, 99],
    'feature_selection__type': ['anova_f']
}

# Define the pipeline
pipeline = Pipeline([
    ('noise_removal', NoiseRemover()),
    ('emoticon_convertion', EmoticonConverter()),
    ('punctuation_removal', PunctuationRemover()),
    ('tokenization', Tokenizer()),
    ('garbageout', GarbageRemover()),
    ('stopword_removal', StopwordRemover()),
    ('lemmatization', Lemmatizer()),
    ('vectorization', Vectorize()),
    #('oversampler', RandomOverSampler(sampling_strategy=oversampling_strategy, random_state=42)),
    #('undersampler', RandomUnderSampler(sampling_strategy=undersampling_strategy, random_state=42)),
    ('smote', SMOTE()),
    ('scaler', Scaler()),
    ('feature_selection', FeatureSelector()),
    ('model', None) #Placeholder for the model to be used
])

# Define the various models and their hyperpameters to include in Gridsearch
model_params = {
    #'Multinomial Naive Bayes': {
    #    **preprocessing_params,
    #    'model': [MultinomialNB()],
    #    'model__alpha': [0.01, 0.1, 1],
    #    'model__fit_prior': [True, False]
    #},
    'LogisticRegression': {
        **preprocessing_params,
        'model': [LogisticRegression()],
        'model__penalty': ['l2'],
        'model__C': [0.001, 0.1, 1, 10],
        'model__solver': ['sag'],
        'model__multi_class': ['multinomial'],
        'model__max_iter': [1000000],
        'model__tol': [0.0001, 0.00001],
        'model__random_state': [42]
    }
}

for model_name, param_grid in model_params.items():
    print(f"Training {model_name}...")
    # Update the pipeline with the classifier
    pipeline.set_params(model=None)  # Reset the classifier
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro', error_score='raise', verbose=2)
    grid_search.fit(X_train, y_train)
    df_gridsearch = pd.DataFrame(grid_search.cv_results_)
    # Write to CSV file
    selected_columns = ['Model','Dataset','Timestamp','mean_fit_time','mean_score_time','change_labels','params','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score','std_test_score','mean_test_score']
    #Create blank results dataframe - only do this once
    #results_df = pd.DataFrame(columns=selected_columns)
    #results_df.to_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/Model_Comparison results.csv', index=False)
    new_data = pd.DataFrame(columns=selected_columns)
    new_data[['mean_fit_time','mean_score_time','params','split0_test_score','split1_test_score',
         'split2_test_score','split3_test_score','split4_test_score','std_test_score','mean_test_score']] = df_gridsearch[['mean_fit_time',
         'mean_score_time','params','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score','std_test_score','mean_test_score']]
    new_data['Model'] = model_name
    new_data['Dataset'] = size
    new_data['Timestamp']= datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    new_data['change_labels'] = label_change
    new_data.to_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/Model_Comparison results.csv', mode='a', header=False, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training LogisticRegression...
Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] END feature_selection__percentile=10, feature_selection__type=anova_f, lemmatization__pos=v, model=LogisticRegression(), model__C=0.001, model__max_iter=1000000, model__multi_class=multinomial, model__penalty=l2, model__random_state=42, model__solver=sag, model__tol=0.0001, scaler__type=robust, smote__k_neighbors=3, smote__random_state=42, tokenization__type=TweetTokenizer, vectorization__max_df=0.25, vectorization__max_features=None, vectorization__min_df=1, vectorization__ngram_range=(1, 1), vectorization__type=count; total time=   1.7s
[CV] END feature_selection__percentile=10, feature_selection__type=anova_f, lemmatization__pos=v, model=LogisticRegression(), model__C=0.001, model__max_iter=1000000, model__multi_class=multinomial, model__penalty=l2, model__r

In [2]:
df_gridsearch = pd.DataFrame(grid_search.cv_results_)
    # Write to CSV file
selected_columns = ['Model','Dataset','Timestamp','mean_fit_time','mean_score_time','params','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score','std_test_score','mean_test_score']
    #Create blank results dataframe - only do this once
    #results_df = pd.DataFrame(columns=selected_columns)
    #results_df.to_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/Model_Comparison results.csv', index=False)
new_data = pd.DataFrame(columns=selected_columns)
new_data[['mean_fit_time','mean_score_time','params','split0_test_score','split1_test_score',
         'split2_test_score','split3_test_score','split4_test_score','std_test_score','mean_test_score']] = df_gridsearch[['mean_fit_time',
         'mean_score_time','params','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score','std_test_score','mean_test_score']]
new_data['Model'] = model_name
new_data['Dataset'] = size
new_data['Timestamp']= datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
new_data.to_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/Model_Comparison results.csv', mode='a', header=False, index=False)

NameError: name 'pd' is not defined

In [238]:
# Train model on 100% of the data

df_train, _ = load_dataset('colab',1,change_labels=True)
X_train, X_test, y_train, y_test = train_test_split(df_train['message'], df_train['sentiment'], test_size=0.2, random_state=42)#, stratify=df_train['sentiment'])
preprocessing_steps = [
                       ('noise_removal', NoiseRemover(switch='off')),
                       ('emoticon_convertion', EmoticonConverter(switch='off')),
                       ('punctuation_removal', PunctuationRemover(switch='off')),
                       ('tokenization', Tokenizer(type='TreebankWordTokenizer')),
                       ('garbageout', GarbageRemover(switch='off')),
                       ('stopword_removal', StopwordRemover()),
                       ('lemmatization', Lemmatizer(pos='v')),
                       ('vectorization', Vectorize(max_df=0.75, max_features=None,
                                                   min_df=1,ngram_range=(1, 1),type='tfidf')),
                       ('smote', SMOTE(k_neighbors=3, random_state=42)),
                       ('feature_selection', FeatureSelector(type='chi2',percentile=75)),
                       ('scaler', Scaler(type='robust')),
                       ]
model = LogisticRegression(C=10, max_iter=100000,multi_class='ovr', solver='liblinear', tol=0.0001, penalty='l2', random_state=42)
pipeline = Pipeline(preprocessing_steps  + [('model', model)])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred)
print("Classification Report for test set:")
print(report)
print('************************')

k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
predictions = cross_val_predict(pipeline, X_train, y_train, cv=k_fold)
report = classification_report(y_train, predictions)
print("Classification Report for CV:\n", report)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Classification Report for test set:
              precision    recall  f1-score   support

        Anti       0.62      0.55      0.59       278
     Neutral       0.48      0.46      0.47       425
        News       0.69      0.82      0.75       706
         Pro       0.81      0.77      0.79      1755

    accuracy                           0.72      3164
   macro avg       0.65      0.65      0.65      3164
weighted avg       0.72      0.72      0.72      3164

************************
Classification Report for CV:
               precision    recall  f1-score   support

        Anti       0.60      0.53      0.56      1018
     Neutral       0.55      0.44      0.49      1928
        News       0.68      0.82      0.74      2934
         Pro       0.80      0.78      0.79      6775

    accuracy                           0.72     12655
   macro avg      

In [ ]:
'''
# Evaluate the pipeline
accuracy = pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

model_stats = pipeline.named_steps['model']

# Obtain the number of features used by the model
attributes = model_stats.__dict__

# Print all attributes
for attr, value in attributes.items():
    print(attr, ":", value)
'''

In [ ]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'G:/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/dawieloots_predict_gridsearch_{timestamp}.csv'
df_gridsearch.to_csv(filename, index=False)

In [ ]:

print(df_gridsearch)
print(grid_search.get_params)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Custom transformer to wrap SMOTE

class ResampleAndFeatureSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, k=1000, score_func=chi2, k_neighbors=5):
        self.k = k
        self.score_func = score_func
        self.k_neighbors = k_neighbors
        self.feature_selector = SelectKBest(score_func=self.score_func, k=self.k)
        
    def fit(self, X, y):
        # Print the shape of X_train before preprocessing
               
        # Select features from the resampled data
        self.feature_selector.fit(X, y)
        
        # Print the shape of X_train after preprocessing
        
        return self

    def transform(self, X):
        # Select features from the input data
        X_selected = self.feature_selector.transform(X)
        return X_selected

# Load the 20 Newsgroups dataset
data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data_subset = {
    'data': data.data[:1000],
    'target': data.target[:1000],
    'target_names': data.target_names
}

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_subset['data'], data_subset['target'], test_size=0.2, random_state=42)

# Define preprocessing steps
preprocessing_steps = [
    # Text preprocessing
    ('vectorizer', TfidfVectorizer()),  # Convert text data into numerical vectors
    ('smote', SMOTE()),
    ('resample_and_feature_select', ResampleAndFeatureSelectTransformer(k=1000, score_func=chi2)),  # Resample and select top k features
    
]

# Define the model
model = KNeighborsClassifier()

# Create the pipeline
pipeline = Pipeline(preprocessing_steps + [('model', model)])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
accuracy = pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

model = pipeline.named_steps['model']

# Obtain the number of features used by the model
attributes = model.__dict__

# Print all attributes
for attr, value in attributes.items():
    print(attr, ":", value)
#n_features = model.n_features_
#print("Number of features used by the model:", n_features)


In [254]:
## Final model building with full trainset
df_train, _  = load_dataset('colab',1,change_labels=False)
X_train = df_train['message']
y_train = df_train['sentiment']

preprocessing_steps =  [
                       ('noise_removal', NoiseRemover(switch='off')),
                       ('emoticon_convertion', EmoticonConverter(switch='off')),
                       ('punctuation_removal', PunctuationRemover(switch='off')),
                       ('tokenization', Tokenizer(type='TreebankWordTokenizer')),
                       ('garbageout', GarbageRemover(switch='off')),
                       ('stopword_removal', StopwordRemover()),
                       ('lemmatization', Lemmatizer(pos='v')),
                       ('vectorization', Vectorize(max_df=0.75, max_features=None,
                                                   min_df=1,ngram_range=(1, 1),type='tfidf')),
                       ('smote', SMOTE(k_neighbors=3, random_state=42)),
                       ('feature_selection', FeatureSelector(percentile=75, type='chi2')),
                       ('scaler', Scaler(type='robust')),
                       ]

model = LogisticRegression(C=10, max_iter=10000,multi_class='ovr', solver='liblinear', tol=0.0001, penalty='l2', random_state=42)
pipeline = Pipeline(preprocessing_steps  + [('model', model)])
pipeline.fit(X_train, y_train)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Pipeline(steps=[('noise_removal', NoiseRemover(switch='off')),
                ('emoticon_convertion', EmoticonConverter(switch='off')),
                ('punctuation_removal', PunctuationRemover(switch='off')),
                ('tokenization', Tokenizer(type='TreebankWordTokenizer')),
                ('garbageout', GarbageRemover(switch='off')),
                ('stopword_removal', StopwordRemover()),
                ('lemmatization', Lemmatizer(pos='v')),
                ('vectorization',
                 Vectorize(max_df=0.75, min_df=1, ngram_range=(1, 1),
                           type='tfidf')),
                ('smote', SMOTE(k_neighbors=3, random_state=42)),
                ('feature_selection',
                 FeatureSelector(percentile=75, type='chi2')),
                ('scaler', Scaler(type='robust')),
                ('model',
                 LogisticRegression(C=10, max_iter=10000, multi_class='ovr',
                                    random_state=42, solver='liblinear'))])

In [173]:
model_stats = pipeline.named_steps['model']

# Obtain the number of features used by the model
attributes = model_stats.__dict__

# Print all attributes
for attr, value in attributes.items():
    print(attr, ":", value)

penalty : l2
dual : False
tol : 0.0001
C : 10
fit_intercept : True
intercept_scaling : 1
class_weight : None
random_state : 42
solver : liblinear
max_iter : 10000
multi_class : ovr
verbose : 0
warm_start : False
n_jobs : None
l1_ratio : None
n_features_in_ : 22738
classes_ : ['Anti' 'Neutral' 'News' 'Pro']
coef_ : [[-0.40087111  3.92462796  0.93506867 ... -0.27201244 -0.27201244
  -0.23904429]
 [-1.42119464 -3.58016736 -0.86139242 ...  1.26184713  1.26184713
   0.88613006]
 [-0.23830293  1.29231991 -0.05515549 ... -0.33987135 -0.33987135
  -0.16240986]
 [ 2.99299461 -1.79103157 -0.19007266 ... -0.80658949 -0.80658949
  -0.68330685]]
intercept_ : [-3.11560453 -0.01949719 -3.87447206 -0.91209656]
n_iter_ : [16 18 15 18]


In [255]:
test_file = '/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/data/test_with_no_labels.csv'
df_test = pd.read_csv(test_file)
X_unseen = df_test['message']

# Predict
y_pred_text = pipeline.predict(X_unseen)
label_map = {
        'News': 2,
        'Pro': 1,
        'Neutral': 0,
        'Anti': -1
    }
#y_pred_int = [label_map[label] for label in y_pred_text]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/dawieloots_predict_{timestamp}.csv'
submission_df = pd.DataFrame()
submission_df['tweetid'] = df_test.tweetid
submission_df['sentiment'] = y_pred_text
submission_df.to_csv(filename, index=False)

In [256]:
submission_df.head()

,tweetid,sentiment
0,169760,1
1,35326,0
2,224985,1
3,476263,0
4,872928,1
